### Basic RAG 
 This implementation of RAG is inspired by the teachings of  Vasanth P sir Yt channel(Neural Hacks with Vasanth).



In [ ]:
from dotenv import load_dotenv ,find_dotenv
load_dotenv(find_dotenv())

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2']='True'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")

In [ ]:
import bs4
from langchain import hub
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:


loader=WebBaseLoader(
    web_path=("https://www.ibm.com/think/topics/transformer-model",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post_content","cmp-container")
        )
    ),
)

docs=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks_data=text_spliter.split_documents(docs)

# Define the model parameters
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf_em_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vecotorstore=FAISS.from_documents(
    documents=chunks_data,
    embedding=hf_em_model
)

retrive_data=vecotorstore.as_retriever()

llm=ChatGroq(
    model="llama3-8b-8192", temperature=0
)

prompt=hub.pull("rlm/rag-prompt")

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain=(
    {"context":retrive_data|format_docs,"question":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()

)



'A transformer model is a type of deep learning model introduced in 2017, which has become fundamental in natural language processing (NLP) and has been applied to various tasks in machine learning and artificial intelligence. It processes input data through a series of layers containing self-attention mechanisms and feedforward neural networks. Unlike recurrent neural networks (RNNs) and convolutional neural networks (CNNs), transformer models process input sequences in parallel, making them highly efficient for training and inference.'

In [ ]:
rag_chain.invoke("what is Transformer model?")

'A transformer model is a type of deep learning model introduced in 2017, which has become fundamental in natural language processing (NLP) and has been applied to various tasks in machine learning and artificial intelligence. It processes input data through a series of layers containing self-attention mechanisms and feedforward neural networks. Unlike recurrent neural networks (RNNs) and convolutional neural networks (CNNs), transformer models process input sequences in parallel, making them highly efficient for training and inference.'